In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import code_step2 as cs

# Load data

In [2]:
importlib.reload(cs)
num_profiles = 300
num_minutes = 60

# Generate 300 stochastic load profiles
all_profiles = np.array([cs.generate_profile(num_minutes=num_minutes) for _ in range(num_profiles)])
print(all_profiles.shape)

in_sample = all_profiles[0:100,:].copy()
out_sample = all_profiles[100:,:].copy()
print(in_sample.shape)
print(out_sample.shape)

(300, 60)
(100, 60)
(200, 60)


# Cvar

In [20]:
importlib.reload(cs)

eps=0.1
cvar_capacity, cvar_problem = cs.cvar(in_sample, eps, verbose=False)
print(f'Offered capacity for FCR-D UP: {cvar_capacity} kW')

Offered capacity for FCR-D UP: 250.19494883498194 kW


# ALSO-X

In [19]:
importlib.reload(cs)

eps=0.1
also_x_capacity, also_x_problem = cs.also_x(in_sample, eps, verbose=False)
print(f'Offered capacity for FCR-D UP: {also_x_capacity} kW')

Iteration: 0
0.2999999999993327 0.5 0 0.5
Iteration: 1
0.14999999999998853 0.25 0 0.25
Iteration: 2
0.07499999999971678 0.25 0.125 0.125
Iteration: 3
0.11249999999968584 0.1875 0.125 0.0625
Iteration: 4
0.09374999999996116 0.1875 0.15625 0.03125
Iteration: 5
0.10312499999997049 0.171875 0.15625 0.015625
Iteration: 6
0.09843749999999242 0.171875 0.1640625 0.0078125
Iteration: 7
0.10078124999999537 0.16796875 0.1640625 0.00390625
Iteration: 8
0.09960937499999378 0.16796875 0.166015625 0.001953125
Iteration: 9
0.10019531249994419 0.1669921875 0.166015625 0.0009765625
Iteration: 10
0.0999023437499839 0.1669921875 0.16650390625 0.00048828125
Iteration: 11
0.10004882812479336 0.166748046875 0.16650390625 0.000244140625
Iteration: 12
0.09997558593748666 0.166748046875 0.1666259765625 0.0001220703125
Iteration: 13
0.10001220703123852 0.16668701171875 0.1666259765625 6.103515625e-05
Offered capacity for FCR-D UP: 240.90098036091675 kW


# Out of sample analysis

In [ ]:
all_out_samples = out_sample.flatten()

all_out_samples-also_x_capacity
print(np.quantile(all_out_samples-also_x_capacity, 0.1))
print(np.quantile(all_out_samples-cvar_capacity, 0.1))

# these values are positive (meaning the probability of reserve shortfall is less than 10% (if my logic is correct))


20.383325193911908
11.085946933434464
